In [1]:
import pandas as pd
ks=pd.read_csv('G:/MINE/SUB/Thesis/kaggle/6.Feature Engineering/DataSet/ks-projects-201801.csv',parse_dates=['deadline','launched'])

In [6]:
ks=ks.query("state!='live'")
ks=ks.assign(outcome=(ks.state=='successful').astype(int))
ks=ks.assign(hour=ks.launched.dt.hour,
            day=ks.launched.dt.day,
            month=ks.launched.dt.month,
            year= ks.launched.dt.year)
num_col=['goal','hour','day','month','year','outcome']
cat_col=['category','currency','country']
from sklearn.preprocessing import LabelEncoder
encoder=LabelEncoder()
encode=ks[cat_col].apply(encoder.fit_transform)
baseline=ks[num_col].join(encode)

In [3]:
baseline.head()

,goal,hour,day,month,year,outcome,country,currency,category
0,1000.0,12,11,8,2015,0,9,5,108
1,30000.0,4,2,9,2017,0,22,13,93
2,45000.0,0,12,1,2013,0,22,13,93
3,5000.0,3,17,3,2012,0,22,13,90
4,19500.0,8,4,7,2015,0,22,13,55


In [7]:
import itertools
interaction=pd.DataFrame(index=ks.index)
for col1,col2 in itertools.combinations(cat_col,2):
    col_name=col1+'_'+col2
    col_value=ks[col1].map(str)+'_'+ks[col2].map(str)
    label_encoder=LabelEncoder()
    interaction[col_name]=label_encoder.fit_transform(col_value)

interBaseline=baseline.join(interaction)
interBaseline.head()

,goal,hour,day,month,year,outcome,category,currency,country,category_currency,category_country,currency_country
0,1000.0,12,11,8,2015,0,108,5,9,1215,1900,18
1,30000.0,4,2,9,2017,0,93,13,22,1047,1630,31
2,45000.0,0,12,1,2013,0,93,13,22,1047,1630,31
3,5000.0,3,17,3,2012,0,90,13,22,1024,1595,31
4,19500.0,8,4,7,2015,0,55,13,22,630,979,31


In [8]:
release=pd.Series(ks.index,index=ks.launched,name='count_7days').sort_index()
count_7days=release.rolling('7d').count()-1
count_7days.index=release.values
count_7days=count_7days.reindex(ks.index)
cout_inter_baseline=interBaseline.join(count_7days)

In [10]:
cout_inter_baseline

,goal,hour,day,month,year,outcome,category,currency,country,category_currency,category_country,currency_country,count_7days
0,1000.0,12,11,8,2015,0,108,5,9,1215,1900,18,1409.0
1,30000.0,4,2,9,2017,0,93,13,22,1047,1630,31,957.0
2,45000.0,0,12,1,2013,0,93,13,22,1047,1630,31,739.0
3,5000.0,3,17,3,2012,0,90,13,22,1024,1595,31,907.0
4,19500.0,8,4,7,2015,0,55,13,22,630,979,31,1429.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
378656,50000.0,2,17,9,2014,0,39,13,22,447,699,31,1482.0
378657,1500.0,3,22,6,2011,0,93,13,22,1047,1630,31,505.0
378658,15000.0,19,1,7,2010,0,93,13,22,1047,1630,31,238.0
378659,15000.0,18,13,1,2016,0,138,13,22,1546,2412,31,1100.0


In [13]:
sameProject=pd.DataFrame(ks[['category','launched']],index=ks.index).sort_values('launched')

In [17]:
def time_difference(series):
    return series.diff().dt.total_seconds()/3600
timedeltas=sameProject.groupby('category').transform(time_difference)
timedeltas=timedeltas.fillna(timedeltas.max())
final_baseline=cout_inter_baseline.join(timedeltas.rename({'launched':'time_since_last_Project'},axis=1))

In [18]:
final_baseline.head()

,goal,hour,day,month,year,outcome,category,currency,country,category_currency,category_country,currency_country,count_7days,time_since_last_Project
0,1000.0,12,11,8,2015,0,108,5,9,1215,1900,18,1409.0,18.606111
1,30000.0,4,2,9,2017,0,93,13,22,1047,1630,31,957.0,5.592778
2,45000.0,0,12,1,2013,0,93,13,22,1047,1630,31,739.0,1.313611
3,5000.0,3,17,3,2012,0,90,13,22,1024,1595,31,907.0,0.635000
4,19500.0,8,4,7,2015,0,55,13,22,630,979,31,1429.0,16.661389


In [19]:
def make_split(df,fraction=.1):
    size=int(len(df)*fraction)
    train=df[:-2*size]
    valid=df[-2*size:-size]
    test=df[-size:]
    return train,valid,test

def train_model(train,valid,test=None,feature=None):
    import lightgbm as lgb
    from sklearn.metrics import roc_auc_score
    if feature is None:
        feature=train.columns.drop('outcome')
    dtrain=lgb.Dataset(train[feature],label=train['outcome'])
    dvalid=lgb.Dataset(valid[feature],label=valid['outcome'])
    param={'num_leaves':64,'objective':'binary','metric':'auc','seed':7}
    num_round=1000
    model=lgb.train(param,dtrain,num_round,valid_sets=[dvalid],verbose_eval=False,early_stopping_rounds=20)
    validPredict=model.predict(valid[feature])
    print('\n'*4)
    print("wait a second to see accuracy on valid sets .. .")
    print("valid accuracy = {}".format(roc_auc_score(valid['outcome'],validPredict)))
    if test is not None:
        print('wait another second to see accuracy on test set .. .')
        print('test accuracy = {}'.format(roc_auc_score(test['outcome'],model.predict(test[feature]))))

In [20]:
train,valid,test=make_split(final_baseline)
train_model(train,valid,test)

[LightGBM] [Info] Number of positive: 107340, number of negative: 193350
[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.013818 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 1553
[LightGBM] [Info] Number of data points in the train set: 300690, number of used features: 13
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.356979 -> initscore=-0.588501
[LightGBM] [Info] Start training from score -0.588501





wait a second to see accuracy on valid sets .. .
valid accuracy = 0.7466774415346332
wait another second to see accuracy on test set .. .
test accuracy = 0.746659055391359
